# 先Filter 在2015/12 ~ 2018/12 存續的ETF

## 先開啟檔案位置，並設定初始時間作為篩選門檻

In [1]:
import csv
from datetime import datetime

#path = r'C:\Users\User\Desktop\ETF分組\CURRENCY + PRE STOCK (第十五組)\第15組'

start_date = datetime.strptime( "2016/01/01" , "%Y/%m/%d")

out_file =  open( r'C:\Users\User\Documents\GitHub\HW1\HW1\data\ETF List Filtered.csv', 'w',newline='')
in_file1 = open( r'C:\Users\User\Documents\GitHub\HW1\HW1\data\Currency ETF List (36).csv', 'r') 
in_file2 = open( r'C:\Users\User\Documents\GitHub\HW1\HW1\data\Preferred Stock ETF List (12).csv', 'r')
        

## 使用CSV庫
## 用DictReader來讀取特定的行數，用法類似字典
## 用writer來寫出檔案

In [2]:
csv_reader = csv.DictReader(in_file1)
csv_writer = csv.writer(out_file)
csv_writer.writerow(['Symbol', 'ETF Name', 'Inception','Type']) #先寫出行名稱

for a_row in csv_reader: #對每一列，進行Dictionary-like的讀取
    symbol = a_row['Symbol']
    etf_name = a_row['ETF Name']
    inception = datetime.strptime( a_row['Inception'], "%Y/%m/%d") 
#寫入
    if inception  <= start_date:
        csv_writer.writerow([symbol, etf_name, inception.date(), 'Currency']) #寫出四行，分別是代號、名稱、創立時間、跟基金類型

In [3]:
csv_reader2 = csv.DictReader(in_file2)
for a_row in csv_reader2:
    symbol = a_row['Symbol']
    etf_name = a_row['ETF Name']
    inception = datetime.strptime( a_row['Inception'], "%Y/%m/%d")
#寫入
    if inception  <= start_date:
        csv_writer.writerow([symbol, etf_name, inception.date(), 'Prefered Stock'])

## 記得關閉檔案

In [4]:
out_file.close()
in_file1.close()
in_file2.close()

# 爬取ETF淨值，爬取網站為ycharts

## 由於該網頁需要帳號密碼，所以需要一組帳密來登入
## 之後可以調用login函數來取得一個對話控制，此允許我們爬取基金淨值

In [5]:
import requests
from lxml import html
import json
import urllib3
import time
def login():
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
    email = "satai1919@gmail.com"
    pas = "FintechMl"
    LOGIN_URL = "https://ycharts.com/login"
    
    session = requests.session()
    
    result = session.get(LOGIN_URL)
    tree = html.fromstring(result.text)
    authenticity_token = list(set(tree.xpath('//input[@name="csrfmiddlewaretoken"]/@value')))[0]
    
    payload = {
        'username': email,
        'password': pas,
        'csrfmiddlewaretoken': authenticity_token
    }
    
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6,zh-CN;q=0.5,fr;q=0.4',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Content-Length': '142',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cookie': '__utmc=69688216; __utmz=69688216.1552357223.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); hblid=JQ0XqOZqwIXsXwT51y8Lx0HAoArzb6aB; _okdetect=%7B%22token%22%3A%2215523572229710%22%2C%22proto%22%3A%22https%3A%22%2C%22host%22%3A%22ycharts.com%22%7D; olfsk=olfsk12322538568271435; _ok=1228-592-10-8601; hubspotutk=7cf8506f03419977c4be0b998e16b46e; __hssrc=1; _cb_ls=1; _cb=B56Z-XDKRC_SC1U7V5; 33e807c05af9078f6b2ed01ced5fc28d5c8f52f4=1; wcsid=dhJeybtiypg1EtEV1y8Lx0Harz6ZJ0AA; _okbk=cd4%3Dtrue%2Cvi5%3D0%2Cvi4%3D1552376476295%2Cvi3%3Dactive%2Cvi2%3Dfalse%2Cvi1%3Dfalse%2Ccd8%3Dchat%2Ccd6%3D0%2Ccd5%3Daway%2Ccd3%3Dfalse%2Ccd2%3D0%2Ccd1%3D0%2C; _okac=f0b90d7b5b31a4a193bbea257a02ab8a; _okla=1; __utma=69688216.990749136.1552357223.1552376476.1552379039.4; __hstc=165832289.7cf8506f03419977c4be0b998e16b46e.1552357225006.1552376478295.1552379041795.4; _cb_svref=null; csrftoken='+authenticity_token+'; ycsessionid=uualzce6xlp27stjyfdee97evws5ollz; __utmt=1; page_view_ctr=51; __utmb=69688216.46.10.1552379039; mp_bd6455515e9730c7dc2f008755a4ddfe_mixpanel=%7B%22distinct_id%22%3A%20%221696fb366da666-05b45b73c1e575-36657105-fa000-1696fb366db7e2%22%2C%22%24device_id%22%3A%20%221696fb366da666-05b45b73c1e575-36657105-fa000-1696fb366db7e2%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__mps%22%3A%20%7B%7D%2C%22__mpso%22%3A%20%7B%7D%2C%22__mpus%22%3A%20%7B%7D%2C%22__mpa%22%3A%20%7B%7D%2C%22__mpu%22%3A%20%7B%7D%2C%22__mpr%22%3A%20%5B%5D%2C%22__mpap%22%3A%20%5B%5D%2C%22%24user_id%22%3A%20%221696fb366da666-05b45b73c1e575-36657105-fa000-1696fb366db7e2%22%7D; __hssc=165832289.42.1552379041795; _chartbeat2=.1552357225125.1552382317031.1.B8gGKsDfweQGDsM3n-BBKPGqB8ZxQ4.40; _oklv=1552382376998%2CdhJeybtiypg1EtEV1y8Lx0Harz6ZJ0AA',
        'Host': 'ycharts.com',
        'Origin': 'https://ycharts.com',
        'Referer': 'https://ycharts.com/login',
        'Upgrade-Insecure-Requests': '1',
    }
    
    r = session.post("https://ycharts.com/login", data=payload, headers = headers, verify=False)
    
    r.status_code
    return session

In [6]:
import pandas as pd
import re
import csv
from bs4 import BeautifulSoup

## 我們先開啟之前篩選過後的ETF檔案，並準備一個輸出檔案
## 並先寫入共三十七個行名稱: 2018/12~2015/12

In [7]:
filename = r"C:\Users\User\Documents\GitHub\HW1\HW1\data\ETF List Filtered.csv"
inputfile = open(filename, "r")
name = "output.csv"
output = open(name, "w+", newline='')
writer=csv.writer(output)
#write first line(2018/12~2015/12)
time_list=['']
for i in range(2018,2015,-1):
    for j in range(12,0,-1):
        time_list.append(str(i)+'/'+str(j))
time_list.append('2015/12')
writer.writerow(time_list)

a = inputfile.readline()

## 接著對每個基金爬去目標網站
## 爬取後有16個頁面，每個頁面的表格都需要用BeautifulSoup來存取出資料

In [8]:
for i in range(34):
    a = inputfile.readline()
    a = a.split(",")

    session = login() #登錄網站
    nav_dict=dict()    #{'2018 Dec.':[31,123.54]} (key是字串，value是list[day,nat])
    for j in range(1,17): # 16 pages in total
        url = "https://ycharts.com/companies/"+a[0]+"/net_asset_value.json?endDate=12/31/2018&pageNum="+str(j)+"&startDate=12/1/2015"
        reqs = session.get(url) 
        rj = json.loads(reqs.text)
        to_deal = rj["data_table_html"]  
        time.sleep(1)
        soup = BeautifulSoup(to_deal, 'lxml')  

        #使用BeautifulSoup解析出table
        col1 = soup.find_all('td',attrs={"class": "col1"})  
        col2 = soup.find_all('td',attrs={"class": "col2"})
        #print(col1[0].text.split())    #格式是這樣 ['Dec.', '31,', '2018']    一個list
        #print(col2[0].text.strip())    #格式是這樣 '123.54' 一個字串並沒有多餘的符號

        #將時間以及NAV處理好後照順序存入list裡面
        date = [ col1[index].text.split() for index in range(len(col1)) ] 
        nav = [ col2[index].text.strip() for index in range(len(col2)) ]

        #對爬取到的每個時間跑迴圈，找出每個月分最後一個交易日，以此找出該基金的月底淨值
        for index in range(len(col1)):
            month=date[index][2]+' '+date[index][0]
            day=int(re.sub(',', ' ', date[index][1]))
            if nav_dict.get(month)==None:
                nav_dict[month]=[day,nav[index]]
            elif day>nav_dict[month][0]:
                nav_dict[month]=[day,nav[index]]
                
    #對第i個基金寫入該基金的名稱以及37個月的月底基金淨值
    nav_list=[a[0]] 
    for item in list(nav_dict.values()):
        nav_list.append(item[1])
    writer.writerow(nav_list)
    print(a[0],'done')        

FXA done
FXB done
FXC done
FXCH done
FXE done
FXY done
FXSG done
FXS done
FXF done
DBV done
UDN done
UUP done
URR done
DRR done
CNY done
INR done
EUFX done
ULE done
YCL done
CROC done
EUO done
YCS done
USDU done
BZF done
CYB done
CEW done
FPE done
SPFF done
PGF done
PGX done
IPFF done
PFF done
PSK done
PFXF done


In [9]:
# 關閉檔案
inputfile.close()
output.close()